# Structure search with Bayesian Optimization

This tutorial demonstrates how to use BO to optimize the atomistic configuration of materials. This can be done by mapping the configurational degrees of freedom to the total energy landscape, and inferring the global minimum of energy. The energy landscapes, refined by iterative data acquisitons, can be employed to visualise low energy configurations (they are chemically interpretable).

<img src="https://gitlab.com/joalof/bigmax_boss_tutorials/-/raw/319e6ba6dbbf37ac456abc6dac7f4de5ef097312/figures/ss_scenarios.png" width="800px"/>

At the outset, the user should consider which **key degrees of freedom** control the energetics of the research problem. In the case of adsorption of organic molecules to the surface, these could be the molecule position or orientation above the surface. The molecular conformation can often be kept fixed. In the case of flexible molecules, we need to conduct a conformer search and minimize the energy with respect to dihedral angles. The degrees of freedom and their bounds define the N-dimensional BO structure search.

BO can proceed on any data source (experiment or computation), any computer code (molecular mechanics or ab initio approaches), on any number of cores (it has an API and command line interface) and can fit a GP surrogate model of any or many materials properties extracted from calculations (energy, band gaps, HOMO/LUMO). Flexible use of external functions is facilitated by the BOSS **user function**, where the user transforms the BO sampling across the defined search domain into an atomistic configuration, the calculation is performed, and the desired value is parsed and returned.

<img src="https://gitlab.com/joalof/bigmax_boss_tutorials/-/raw/319e6ba6dbbf37ac456abc6dac7f4de5ef097312/figures/user_fn.png" width="800px"/>


## Molecular conformer structure search

Let us consider the problem of alanine conformational structure search, as shown in the image above. We identify 4 degrees of freedom, described by dihedral angles `[d1, d2, d3, d4]`. We will use BO to refine a GPR model mapping the state vector of dihedral angles to conformer energy `E`, as computed by an external total energy code. **Our objective is to find the state vector (configuration) that minimizes the total energy.**

We will first consider a 2D conformer search (other variables kept fixed) and analyse the results, then proceed with a 4D conformer search. <br>
To begin, let's load the required python packages.

In [ ]:
import numpy as np
import GPy
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
%matplotlib inline

from boss.bo.bo_main import BOMain
from boss.pp.pp_main import PPMain

import pylab
pylab.show()

In this example, we used the AMBER code and `gaff` force field to compute the total energy for each sampled configuration. To simplify this tutorial, we construted an **AMBER emulator function** to simulate AMBER energies without the code itself. We did this by building a GPR generative model based on AMBER data, then encoding it in an AMBER emulator function.

In [ ]:
# Here we load AMBER data from a file and build an GPR model.
def load_model(filename):
    """Recreates a GPR model from saved parameters and data. """

    # load saved data
    data = np.load(filename)
    dim = data["X"].shape[1]

    # create kernel and mean functions
    kernel = GPy.kern.StdPeriodic(input_dim=dim, ARD1=True, ARD2=True)
    mean_func = GPy.mappings.Constant(dim, 1)

    # create model
    model = GPy.models.GPRegression(
        data["X"], data["Y"], kernel=kernel, mean_function=mean_func
    )

    # set model params
    model[:] = data["params"]
    model.fix()
    model.parameters_changed()

    return model

# Here, we define the AMBER emulator for a 2D structure search in d1-d4

!wget https://gitlab.com/joalof/bigmax_boss_tutorials/-/raw/319e6ba6dbbf37ac456abc6dac7f4de5ef097312/data/model_2D_E0.npz
AMBER_emulator = load_model("model_2D_E0.npz")

# Here, we define the AMBER emulator for a 4D structure search in d1-d2-d3-d4
#!wget https://gitlab.com/joalof/bigmax_boss_tutorials/-/raw/319e6ba6dbbf37ac456abc6dac7f4de5ef097312/data/model_4D_E0.npz
#AMBER_emulator = load_model("model_4D_E0.npz")

# Here, we define the utility function that retrieves data from the AMBER emulator.
def f(X):
    return AMBER_emulator.predict(np.atleast_2d(X))[0]

## Running BO

Once more, we need to set up the BO run by defining the:

* `bounds` of the search domain: the number of [lower, upper] bound pairs in the vector will tell BOSS that this is a 2D search.
* `yrange` of the expected function
* `kernel`: since all dihedral angles are 360-periodic, we can set the kernel to *standard periodic*
* number of initial points `initpts` and BO iterations `iterpts`

After setting BO, we are ready to run!

In [ ]:
# Here, we define the BO run.
#bounds = [[-0.0, 360.0]] * 2   # It can be done like this, or explicitly below
bounds = [[0.0, 360.0], [0.0, 360.0]]

bo = BOMain(
    f,
    bounds,
    yrange=[0, 1],
    kernel="stdp",
    initpts=5,
    iterpts=40,
)

# Here we perform BO, it should be a minute or two
res = bo.run()

We immediately run post-processing to check results. We set additional options to:

* `pp_iters`: postprocess only BO steps 0, 10, 20 and 40 for convenience
* `pp_model_slice`: explicitly request a model slice in [dim1, dim2, Npts], which is useful to visualise N-dimensional models

In [ ]:
pp = PPMain(res, pp_models=True, pp_iters=[0, 10, 20, 40], pp_model_slice=[1,2, 50])
pp.run()

## Results and Analysis
We start by checking **data acqusition** plots. Since we have 2 dimensions, the lower plot now features the sampling locations in both dihedrals. Amber energy values are all positive, and a global minimum was clearly idenfitifed. On the right, the **hyperparameter plot** indicates that the model has also converged.


In [ ]:
# Let's inspect model data in BOSS

# figure size in inches optional
rcParams['figure.figsize'] = 18, 14

# read images
acq = mpimg.imread('postprocessing/acquisition_locations.png')
param = mpimg.imread('postprocessing/hyperparameters.png')

# display images
fig, ax = plt.subplots(1,2)
ax[0].imshow(acq)
ax[1].imshow(param)

Let us review the potential energy surface (PES) landscapes simulated by GPR after 10, 20 and 40 BO points. We can see that **the global mininum was found with few datapoints**, but more sampling is needed to converge the energy landscapes.

In [ ]:
# Let's inspect the d1-d4 2D model fitting for alanine

# figure size in inches optional
rcParams['figure.figsize'] = 18, 14

# read images
it1 = mpimg.imread('postprocessing/graphs_models/it0010_npts0015.png')
it3 = mpimg.imread('postprocessing/graphs_models/it0020_npts0025.png')
it5 = mpimg.imread('postprocessing/graphs_models/it0040_npts0045.png')

# display images
fig, ax = plt.subplots(1,3)
ax[0].imshow(it1)
ax[1].imshow(it3)
ax[2].imshow(it5)

# Exercise:
Re-do the conformer search with **all 4 degrees of freedom** by uncommenting code above. Note that you have to load the 4D dataset for the AMBER emulator function, and adjust the bounds for 4 dimensions instead of 2. You can execute the BO run with the same settings. In 4D, BO should take a couple of min.

Re-plot all the graphs above. In a 4D structure search, domain volume to cover by sampling is much bigger than in 2D, so the same amount of sampling results in poorer models.

* How does the convergence look for the global minimum prediction and the model hyperparameters?
* What about the energy landscapes?

When post-processing, you can now choose to slice the 4D surrogate model in the plane of any dihedral pair with the `pp_model_slice` keyword. The cross-section is always extracted in the plane of the global minimum solution (you can choose other planes via keywords). Once the BO run is complete, you can rerun the post-processing in different ways, but note that the `postprocessing` directory will be overwritten unless you rename it.

* Dihedral variable d7 has a 120deg symmetry which can be accounted for with a shorter domain boundary - is this visible in the energy landscapes?
* The 2D model was produced for d1-d4 pair, which dominates conformer energetics. If you slice the 4D model in the d1-d4 plane, does it look similar?